# Módulo # 4 - Big Data

## Tarea # 3 
## Autor: Jose Martinez

# Datos de Entrada

## Abandono de Banco

Este conjunto de datos contiene detalles de los clientes de un banco y la variable objetivo es una variable binaria que refleja el hecho de si el cliente dejó el banco (cerró su cuenta) o si continúa siendo un cliente.

### Features

- `RowNumber`: Número de fila (Int)
- `CustomerId`: Identificador del cliente (Int)
- `Surname`: apellido del cliente (String)
- `CreditScore`: puntaje de crédito del cliente (Number)
- `Geography`: geografía del cliente (String)
- `Gender`: Sexo del cliente (String)
- `Age`: Edad del cliente (Int)
- `Tenure`: Número de años que el cliente ha estado en el banco (Int)
- `Balance`: estado de cuenta del cliente (Float)
- `NumOfProducts`: numero de productos del cliente (Int)
- `HasCrCard`: tiene tarjeta de crédito (Bool)
- `IsActiveMember`: es un miembro activo (Bool)
- `EstimatedSalary`: salario estimado del cliente (Float)

### Objetivo Predictivo

- `Exited`: si el cliente abandonó el banco (Bool)

In [ ]:
import findspark
findspark.init('/usr/lib/python3.7/site-packages/pyspark')

from pyspark.sql.types import (StringType, IntegerType, FloatType, 
                                StructField, StructType)

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Bigdata: Tarea 3") \
    .config("spark.driver.extraClassPath", "postgresql-42.2.14.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.2.14.jar") \
    .getOrCreate()

# Define the schema of the dataframe
churn_df = spark \
    .read \
    .format("csv") \
    .option("path", "churn_modelling.csv") \
    .option("header", True) \
    .schema(StructType([
                StructField("RowNumber", IntegerType()),
                StructField("CustomerId", IntegerType()),
                StructField("Surname", StringType()),
                StructField("CreditScore", IntegerType()),
                StructField("Geography", StringType()),
                StructField("Gender", StringType()),
                StructField("Age", IntegerType()),
                StructField("Tenure", IntegerType()),
                StructField("Balance", FloatType()),
                StructField("NumOfProducts", IntegerType()),
                StructField("HasCrCard", IntegerType()),
                StructField("IsActiveMember", IntegerType()),
                StructField("EstimatedSalary", FloatType()),
                StructField("Exited", IntegerType())])) \
    .load()

# Print the schema of the dataframe
churn_df.printSchema()
churn_df.show()

# Preprocesamiento de datos

In [ ]:
from pyspark.sql.functions import when

# Se hace un primer filtrado para eliminar los registros que no tienen
# información valiosa para calcular el modelo. Como lo son los registros
# RowNumber, CustomerId, Surname, Geography. 

columns_kept = ['CreditScore', 'Gender', 'Age', 'Tenure', 
                'Balance', 'NumOfProducts', 'HasCrCard',
                'IsActiveMember', 'EstimatedSalary', 'Exited']
                
selected_columns_df = churn_df.select(columns_kept)

# Change Gender to binary int
selected_columns_df = selected_columns_df.withColumn('Gender',
                                                     when(selected_columns_df.Gender == 'Male', 1)
                                                     .when(selected_columns_df.Gender == 'Female', 0)
                                                     .otherwise(selected_columns_df.Gender))
selected_columns_df = selected_columns_df .withColumn('Gender', selected_columns_df['Gender'].cast(IntegerType()))

selected_columns_df.show()

## Gráficos y Estadísticas Descriptivas

In [ ]:
# Imprimimos información de los datos para verificar que no hay ningún
# problema con los datos.
selected_columns_df.describe(['CreditScore', 'Gender', 'Age', 'Tenure']).show()
selected_columns_df.describe(['Balance', 'NumOfProducts', 'HasCrCard']).show()
selected_columns_df.describe(['IsActiveMember', 'EstimatedSalary', 'Exited']).show()

In [ ]:

# Para realizar operaciones más detalladas es necesario expresar las filas originales en vectores
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['CreditScore', 'Gender', 'Age', 'Tenure', 
                'Balance', 'NumOfProducts', 'HasCrCard',
                'IsActiveMember', 'EstimatedSalary'],
    outputCol='features')

vector_df = assembler.transform(selected_columns_df)
vector_df = vector_df.select(['features', 'Exited'])
vector_df.show()

In [ ]:
# Con la representación de vectores podemos calcular correlaciones
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt

pearson_matrix = Correlation.corr(vector_df, 'features').collect()[0][0]

sns.heatmap(pearson_matrix.toArray(), annot=True, fmt=".2f", cmap='viridis')

## Imputacion de valores faltantes

El dataset fue revisando previamente para ver si existen valores faltantes. En este se cuanta con la fortuna de que no existen valores faltantes, por lo que no es necesario realizar ninguna acción. 

## Normalización / Estandarización de Datos

In [ ]:
from pyspark.ml.feature import StandardScaler, Normalizer

standard_normalizer = Normalizer(inputCol='features', outputCol='normFeatures')
normalize_df = standard_normalizer.transform(vector_df)
normalize_df.show()

standard_scaler = StandardScaler(inputCol='normFeatures', outputCol='scaledFeatures')
scale_model = standard_scaler.fit(normalize_df)

scaled_df = scale_model.transform(normalize_df)
scaled_df.show()

## Escritura a base de datos.

In [ ]:
# TODO: Convert vector to column
from pyspark.ml.functions import vector_to_array

pre_df =scaled_df.withColumn('features', vector_to_array('scaledFeatures'))

pre_df.show()
# Almacenar el conjunto de datos limpio en la base de datos
scaled_df \
    .write \
    .format("jdbc") \
    .mode('overwrite') \
    .option("url", "jdbc:postgresql://172.17.0.1:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "tarea3") \
    .save()

# Entrenamiento del modelo

In [ ]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.17.0.1:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "tarea3") \
    .load()

df.show()

## Uso de protocolo K-fold cross validation

In [ ]:
# TODO: Split the data into training and test sets (70 % training, 30 % test)


## Modelo 1: Regresión Logística 

In [ ]:
# Nótese que no se hace partición de datos de entrenamiento (ejercicio posterior).
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(
    inputCols=['Features[0]', 'Features[1]', 'Features[2]', 'Features[3]', 
               'Features[4]', 'Features[5]', 'Features[6]',
               'Features[7]', 'Features[8]'],
    outputCol='Features')

vector_df = assembler.transform(selected_columns_df)
vector_df = vector_df.select(['Features', 'Exited'])
vector_df.show()

regression = LinearRegression(featuresCol='Features', labelCol='Exited')
regression_model = regression.fit(df)

print('Pesos: {}\n b: {}'.format(regression_model.coefficients, regression_model.intercept))

print('RMSE: {} r2: {}'.format(
    regression_model.summary.rootMeanSquaredError,
    regression_model.summary.r2))

scaled_df.describe().show()

## Modelo 2: Random Forest

# Evaluación del conjunto de validación

## Evaluación y almacenado de modelo 1

In [ ]:
# Guardar en base de datos

## Evaluación y almacenado de modelo 1

In [ ]:
# Guardar en base de datos

## Analisis de resultados

In [ ]:
# PORQUE DIO BIEN? 